In [ ]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy

In [ ]:
# Загрузка данных
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Точка разбиения для тренировочного и тестового наборов
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()
train_test_global_time_split_idx = events["started_at_month"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# Количество уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
common_users = pd.Index(users_train).intersection(pd.Index(users_test))
cold_users = pd.Index(users_test).difference(pd.Index(users_train))

# Удаление редких айтемов (с которыми взаимодействовало менее 2 пользователей)
item_activity = events_train.groupby('item_id')['user_id'].nunique().reset_index(name='user_count')
items_to_keep = item_activity[item_activity['user_count'] >= 2]['item_id']
events_train_filtered = events_train[events_train['item_id'].isin(items_to_keep)]

In [ ]:
items_to_keep.head(10)

In [ ]:
events_train_filtered.head(10)

In [ ]:
# Получаем данные для рекомендаций
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train_filtered[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

In [ ]:
# Инициализация и обучение модели SVD
svd_model = SVD(n_factors=100, random_state=0)
svd_model.fit(surprise_train_set)

In [ ]:
# Получаем предсказания для тестовой выборки
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))
svd_predictions = svd_model.test(surprise_test_set)

In [ ]:
# Оценка рекомендаций
rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)

In [ ]:
# Выводим результаты
print(f"RMSE для SVD модели после удаления редких айтемов: {rmse}")
print(f"MAE для SVD модели после удаления редких айтемов: {mae}")

# Сравнение с предыдущими результатами
# Предыдущие RMSE и MAE (из вашего исходного кода)
previous_rmse = 1.25  # Замените на актуальное значение из вашего исходного кода
previous_mae = 0.99  # Замените на актуальное значение из вашего исходного кода

print(f"Изменение RMSE: {rmse - previous_rmse}")
print(f"Изменение MAE: {mae - previous_mae}")